In [1]:
import numpy as np
import os
import pandas as pd

In [2]:
path = os.getcwd()
dataset_path = path + '\\dataset'
dataset_path

'D:\\JupyterNotebook\\Numpy_practice\\第八次作业\\dataset'

In [3]:
labels = ['环境', '计算机', '交通', '教育', '经济', '军事', '体育', '医药', '艺术', '政治']

In [4]:
# 加载训练集 并进行分词
import jieba
def load_dataset():
    folders = os.listdir(dataset_path)
    data_list = []
    class_list = []
    level = 0
    for folder in folders:
        files = os.listdir(dataset_path + '\\' + folder)
        txt = ""
        for file in files:
            with open(dataset_path + '\\' + folder + '\\' + file, encoding='ANSI', errors='ignore') as f:
                lines = f.readlines()
                for line in lines:
                    txt += line.strip()
                f.close()
            txt += '。'
        words = jieba.lcut(txt)
        word_list = [t for t in words if t not in stopwords and len(t) > 1 and len(t) < 8]
        data_list.append(word_list)
        class_list.append(level)
        level += 1
    return data_list, class_list

In [5]:
# 读取stopwords
def load_stop():
    f = open('stopword.txt', encoding='utf-8')
    stopwords = f.read().split('\n')
    f.close()
    return stopwords

In [6]:
stopwords = load_stop()
len(stopwords)

1537

In [7]:
data_list, class_list = load_dataset()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\OYL\AppData\Local\Temp\jieba.cache
Loading model cost 0.996 seconds.
Prefix dict has been built successfully.


In [8]:
len(data_list), len(class_list)

(10, 10)

In [9]:
# 创建词库 这里就是直接把所有词去重后，当作词库
def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)

In [10]:
myVocabList = createVocabList(data_list)

In [11]:
len(myVocabList)

68563

In [12]:
# 文本词向量。词库中每个词当作一个特征，文本中就该词，该词特征就是1，没有就是0
def trainNB(vocabList):
    returnVec = []
    pclass = []
    word_sum = 0
    word_total = []  # 总的单词（有重复）
    for data in data_list:
        pclass.append(len(data))
        word_sum += len(data)
        word_total += data
    for i, val in enumerate(pclass):
        pclass[i] = val*1.0 / word_sum
    
    word_sum_vocabList = len(vocabList)
    for word in vocabList:
        Vec = []
        word_cnt_total = word_total.count(word) # 所有训练集中词项出现的总次数
        for data in data_list:
            word_cnt = data.count(word)         # 该类下词项出现的总次数
            Vec.append( (word_cnt + 1.0) / (word_cnt_total + word_sum_vocabList) )
        returnVec.append(Vec)
        
    return pclass, returnVec

In [13]:
pclass, pwordvec = trainNB(myVocabList)

In [14]:
print(pclass)

[0.04777550061550742, 0.07122035502400563, 0.08475478218110233, 0.04043151195903129, 0.06887115434164283, 0.11291151007293468, 0.06158442218925295, 0.19657841867639803, 0.13017603006976874, 0.18569631487035612]


In [15]:
print(pwordvec[0])

[1.4584913365614608e-05, 1.4584913365614608e-05, 1.4584913365614608e-05, 2.9169826731229216e-05, 1.4584913365614608e-05, 1.4584913365614608e-05, 1.4584913365614608e-05, 1.4584913365614608e-05, 1.4584913365614608e-05, 1.4584913365614608e-05]


In [16]:
def load_test():
    test_path = path + '\\testdata'
    files = os.listdir(test_path)
    test_data_list = []
    for file in files:
        txt = ''
        with open(test_path + '\\' + file, encoding='ANSI', errors='ignore') as f:
            lines = f.readlines()
            for line in lines:
                txt += line.strip()
            f.close()
        words = jieba.lcut(txt)
        word_list = [t for t in words if t not in stopwords and len(t) > 1 and len(t) < 8]
        test_data_list.append(word_list)
    return files, test_data_list

In [17]:
test_names, test_data_list = load_test()

In [18]:
pclass = [i * 100 for i in pclass]

In [26]:
pclass

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [19]:
def predict(test_list):
    pvec = pclass
    for test_word in test_list:
        if test_word not in myVocabList:
            continue
        word_index = myVocabList.index(test_word)
        for i in range(10):
            pvec[i] *= pwordvec[word_index][i] * 10
    return pvec

In [20]:
predict_vec = []
predict_class = []

for test_data in test_data_list:
    pvec = predict(test_data)
    predict_vec.append(pvec)
    predict_class.append(pvec.index(max(pvec)))

In [21]:
predict_vec[:2], predict_class[:2]

([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]],
 [1, 1])

In [22]:
predict_ans = pd.DataFrame({'name': test_names, 'predict_class': predict_class})

In [23]:
predict_ans.head()

,name,predict_class
0,101.txt,1
1,1010.txt,1
2,10100.txt,0
3,10101.txt,0
4,10102.txt,0


In [24]:
predict_ans.describe

<bound method NDFrame.describe of           name  predict_class
0      101.txt              1
1     1010.txt              1
2    10100.txt              0
3    10101.txt              0
4    10102.txt              0
..         ...            ...
631    995.TXT              0
632    996.TXT              0
633    997.TXT              0
634    998.TXT              0
635    999.TXT              0

[636 rows x 2 columns]>

In [25]:
predict_ans['predict_class'].unique()

array([1, 0], dtype=int64)